## INFOH515 Pyspark code
## Author: Gianluca Bontempi
## Pyspark implementation of the union/intersection operations in the INFOH515 slides "Map-reduce analytics" 

In [2]:
import numpy as np
import pwd
import getpass
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum

# create an instance of SparkSession
spark=SparkSession.builder.appName('s.com').getOrCreate()
sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 14:13:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/17 14:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/17 14:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/17 14:13:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [68]:
Table1=[('Gianluca','M'), ('Lucia','F'),('Gaia', 'F'),('Mattia','M')]

rdd1 = sc.parallelize([(row[0]+row[1], (row[0],row[1])) for  a,row in enumerate(Table1)])

Table2=[('Gianluca','M'), ('Stijn','M'),('Gaia','F')]
rdd2 = sc.parallelize([(row[0]+row[1], (row[0],row[1])) for  a,row in enumerate(Table2)])

In [69]:
rdd1.collect()

[('GianlucaM', ('Gianluca', 'M')),
 ('LuciaF', ('Lucia', 'F')),
 ('GaiaF', ('Gaia', 'F')),
 ('MattiaM', ('Mattia', 'M'))]

In [70]:
rdd=rdd1.union(rdd2)
rdd.collect()

[('GianlucaM', ('Gianluca', 'M')),
 ('LuciaF', ('Lucia', 'F')),
 ('GaiaF', ('Gaia', 'F')),
 ('MattiaM', ('Mattia', 'M')),
 ('GianlucaM', ('Gianluca', 'M')),
 ('StijnM', ('Stijn', 'M')),
 ('GaiaF', ('Gaia', 'F'))]

## Union

In [71]:
rdd.reduceByKey(lambda x, y: x).map(lambda x: x[1]).collect()

[('Gianluca', 'M'),
 ('Stijn', 'M'),
 ('Lucia', 'F'),
 ('Gaia', 'F'),
 ('Mattia', 'M')]

## Intersection

In [72]:

def mapf(x):
    if len(x)>2:
        return((x[0],x[1]))
rdd2=rdd.reduceByKey(lambda x, y: x+y)
rdd2.mapValues(lambda x: mapf(x)).filter(lambda x: x[1] != None).map(lambda x:x[1]).collect()

[('Gianluca', 'M'), ('Gaia', 'F')]

## Difference

In [82]:
Table1=[('Gianluca','M'), ('Lucia','F'),('Gaia', 'F'),('Mattia','M')]

rdd1 = sc.parallelize([(row[0]+row[1], ('R', (row[0],row[1]))) for  a,row in enumerate(Table1)])

Table2=[('Gianluca','M'), ('Stijn','M'),('Gaia','F')]
rdd2 = sc.parallelize([(row[0]+row[1], ('L', (row[0],row[1]))) for  a,row in enumerate(Table2)])
rdd=rdd1.union(rdd2)
rdd.collect()

[('GianlucaM', ('R', ('Gianluca', 'M'))),
 ('LuciaF', ('R', ('Lucia', 'F'))),
 ('GaiaF', ('R', ('Gaia', 'F'))),
 ('MattiaM', ('R', ('Mattia', 'M'))),
 ('GianlucaM', ('L', ('Gianluca', 'M'))),
 ('StijnM', ('L', ('Stijn', 'M'))),
 ('GaiaF', ('L', ('Gaia', 'F')))]

In [136]:
def count(tup, value):
    cnt=0
    for item in tup: 
        if item == value:
            cnt=+1
    return(cnt)


def mapf(x):
    if (len(x)>2):
        if  ((count(x,'R')==1) & (count(x,'L')==1)) :
            return(x[1])


rdd2=rdd.reduceByKey(lambda x, y: x+y)
rdd3=rdd2.mapValues(lambda x: mapf(x)).filter(lambda x: x[1] != None).map(lambda x:x[1])
rdd3.collect()

[('Gianluca', 'M'), ('Gaia', 'F')]